## Imports

In [ ]:
%load_ext lab_black

import h5py
import os

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm

from typing import Dict, List, Optional, Tuple
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    cnn,
)

colors = pc.qualitative.D3
colors_seq = pc.sequential.dense
angstrom = "\u212B"
Theta = "\u0398"

# Load Data

In [ ]:
# ## Convert lamp data ##

# ddir = r"E:\atully\arpes_data\2023_February\Lamp"

# # STEP 1 ##
# # Convert ibw to hdf5
# fn = "Img7_Lamp_g_kw.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # Check conversion worked
# data, theta, energy = HDF5_loader.load_hdf5(
#     ddir, "Img7_Lamp_g_kw.h5"
# )  # load data from hdf5
# data.shape, theta.shape, energy.shape

In [ ]:
# ## Average datasets
# files = []
# files = ["Img55.h5", "Img56.h5"]

# def average_imgs(ddir, files):
#     all_vals = []
#     for file in files:
#         data, theta, energy = loading_functions.load_hdf5(ddir, file)
#         all_vals.append(analysis_functions.limit_dataset(theta, energy, data, xlim, ylim))

#     x_1, y_1, d_1 = all_vals[0]
#     x_2, y_2, d_2 = all_vals[1]
#     data_avg = np.mean([d_1, d_2], axis=0)
#     return x_1, y_1, data_avg
# # print(data_avg.shape)

In [ ]:
def average_imgs(ddir, files, new_filename=None, xlim=None, ylim=None):
    all_vals = []
    for file in files:
        data, theta, energy = loading_functions.load_hdf5(ddir, file)
        all_vals.append(
            analysis_functions.limit_dataset(theta, energy, data, xlim=xlim, ylim=ylim)
        )

    x_vals, y_vals, d_vals = zip(*all_vals)
    data_avg = np.mean(d_vals, axis=0)

    # new_data = data_avg

    # new_fn = os.path.join(ddir, new_filename)

    # with h5py.File(
    #     new_fn, "w"
    # ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
    #     f["data"] = new_data.T
    #     axes_names = [
    #         "angles",
    #         "energies",
    #     ]  # Change these to match your axes labels
    #     axes = [theta, energy]
    #     for axis, name in zip(axes, axes_names):
    #         f[name] = np.atleast_2d(axis).T
    #     entry_group = f.require_group("entry1")
    #     entry_group["ScanValues"] = np.atleast_2d(phi_or_time).T
    # return new_fn
    return x_vals[0], y_vals[0], data_avg

# Initial Parameters

In [ ]:
# title = f"C<sub>60</sub> Pumped Dispersion"
# xaxis_title=f"k<sub>x</sub> [{angstrom}<sup>-1</sup>]",
xaxis_title = f"{Theta}"
yaxis_title = f"E<sub>K</sub> (eV)"

In [ ]:
# time_zero = 37.79  # May13
time_zero = 37.8  # May14

In [ ]:
print(f"Positive delay: {np.round(tr_functions.mm_to_ps(37.85, time_zero), 2)} ps")

# Single Dataset

In [ ]:
ddir = r"E:\atully\arpes_data\2023_May\May13\C60\Imgs"

## HS Scans
file_dict = {
    "Img49.h5": "G-K, 0.4 ps",  # CT1  --> img58 is a duplicate of img49 for higher stats, but also to compare with img58 to make sure nothing has changed
    "Img48.h5": "G-K, zero delay",  # CT2
    "Img50.h5": "G-K, -5 ps",  # CT1
    "Img51.h5": "G-K, -5 ps",  # CT2
    "Img52.h5": "G-K, zero delay",  # CT1/CT2
    "Img53.h5": "K'-G, 0.4 ps",  # CT1
    "Img54.h5": "K'-G, zero delay",  # CT2
    "Img55.h5": "K-M-K, 0.4 ps",  # CT1  --> img56 is a dupicate of img55 for higher stats
    "Img57.h5": "K-M-K, -5 ps",  # CT1
}

In [ ]:
## FFT + Limit Data ##

ddir = r"E:\atully\arpes_data\2023_May\May13\C60\Imgs"

## HS of G - K (pos delay)
# filename = "Img49.h5"
# filename = "Img72.h5"
# filename = "Img74.h5"
# title = "G - K (0.4 ps)"

## HS at G (pos delay)
filename = "Img78.h5"
title = "G (0.4 ps)"

# filename = "Img58.h5"
# filename = "Img59.h5"  # K'-G at pos delay; mystery cone?
# filename = "Img60.h5"  # K'-G at neg delay; mystery cone?
# filename = "Img61.h5"  # K'-G with pump blocked; mystery cone?
# filename = "Img62.h5"  # centered at G with pump blocked; mystery cone?
# filename = "Img64.h5"  # centered at G with pump blocked, new sample spot; mystery cone

data, theta, energy = HDF5_loader.load_hdf5(ddir, filename)  # load data from hdf5

xlim = (-18, 16)
ylim = None
ylim = (energy[57], energy[1007])  # get rid of zero padding on datasets
ylim = (energy[57], 2.5)

f_data = filter_functions.fft2d_mask(data, plot=False)

x_2d, y_2d, d_2d = analysis_functions.limit_dataset(
    theta,
    energy,
    f_data,
    xlim=xlim,
    ylim=ylim,
)

d_2d = analysis_functions.norm_data(d_2d)

# Averaged Images

In [ ]:
ddir = r"E:\atully\arpes_data\2023_May\May13\C60\Imgs"
files = []

files = ["Img55.h5", "Img56.h5", "Img81.h5"]  # K-M-K' pos delay
title = "K - M - K' (0.4 ps)"

# files = ["Img49.h5", "Img72.h5", "Img74.h5"]  # G-K pos delay
# title = "G - K (0.4 ps)"

theta, energy, data = average_imgs(ddir, files)  # "avg_imgs_55_56.h5"

In [ ]:
xlim = (-18, 16)
ylim = None
ylim = (energy[57], energy[1007])  # get rid of zero padding on datasets
ylim = (energy[57], 2.5)

f_data = filter_functions.fft2d_mask(data, plot=False)

x_avg, y_avg, d_avg = analysis_functions.limit_dataset(
    theta,
    energy,
    f_data,
    xlim=xlim,
    ylim=ylim,
)

d_avg = analysis_functions.norm_data(d_avg)

# Stitched Datasets

In [ ]:
## Stitch and Average Datasets ##

files = []
files = ["Img49.h5", "Img53.h5"]
title = "K' - G - K (0.4 ps)"

filename = files[0]
data1, theta1, energy1 = HDF5_loader.load_hdf5(ddir, filename)  # load data from hdf5

filename = files[1]
data2, theta2, energy2 = HDF5_loader.load_hdf5(ddir, filename)  # load data from hdf5


xlim = (-18, 16)
# xlim = None
# ylim = None
# ylim = (energy[57], energy[1007])  # get rid of zero padding on datasets
# ylim = (energy[57], 2.5)

f_data1 = filter_functions.fft2d_mask(data1, plot=False)
f_data2 = filter_functions.fft2d_mask(data2, plot=False)

x1, y1, dataslice1 = analysis_functions.limit_dataset(
    theta1,
    energy1,
    f_data1,
    xlim=xlim,
    ylim=(energy1[57], 2.5),
)

x2, y2, dataslice2 = analysis_functions.limit_dataset(
    theta2,
    energy2,
    f_data2,
    xlim=xlim,
    ylim=(energy2[57], 2.5),
)

xs, ys, ds = tr_functions.stitch_and_avg(
    x1,
    y1,
    dataslice1,
    x2,
    y2,
    dataslice2,
    no_avg=False,
)

# Plot Data

In [ ]:
## Plot Data ##
# x, y, data = x_2d, y_2d, d_2d  # single image
# x, y, data = xs, ys, ds  # stitched image
x, y, data = x_avg, y_avg, d_avg  # averaged image

fig = tr_functions.thesis_fig(
    # title=f"{file_dict[filename]}",
    # title=f"{filename}",
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(x=x, y=y, z=analysis_functions.norm_data(data), coloraxis="coloraxis")
)

fig.update_coloraxes(cmin=0, cmax=0.3)

fig.show()

In [ ]:
## Get and Plot 1D Data --> EDC ##
xlim = None
ylim = None

fig = tr_functions.thesis_fig(
    title=f"EDC",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=1000,
)

y_1d, col = tr_functions.get_1d_x_slice(x=x, y=y, data=data, ylims=ylim, x_range=xlim)

fig.add_trace(go.Scatter(x=y_1d, y=col, line=dict(color=colors[0])))

fig.show()

## Get and Plot 1D Data --> MDC ##
xlim = None
ylim = (2.05, 2.2)

fig = tr_functions.thesis_fig(
    title=f"MDC",
    xaxis_title=xaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=1000,
)

x_1d, row = tr_functions.get_1d_y_slice(x=x, y=y, data=data, xlims=xlim, y_range=ylim)

fig.add_trace(go.Scatter(x=x_1d, y=row, line=dict(color=colors[0])))

fig.show()

In [ ]:
x, data = x_1d, row
offset_type = "constant"

## Offset
c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

# Fit for 2 peaks
gauss1 = fitting_functions.make_gaussian(num="A_", amplitude=1, center=-7, sigma=0.1)
gauss2 = fitting_functions.make_gaussian(num="B_", amplitude=1, center=5, sigma=0.1)
gauss3 = fitting_functions.make_gaussian(num="C_", amplitude=1, center=12, sigma=0.1)

## Full model
# full_model = gauss1 + gauss2 + offset
full_model = gauss1 + gauss2 + gauss3 + offset

params = full_model.make_params()
params["iB__center"].max = 7.4

fit = full_model.fit(data, x=x, params=params)
# fit.plot()


## Plot Fit ##
fig = tr_functions.thesis_fig(
    title=f"MDC",
    xaxis_title=xaxis_title,
    yaxis_title="Residuals",
    equiv_axes=False,
    gridlines=False,
)
fig.add_trace(go.Scatter(x=x, y=row, name="data", line=dict(color=colors[0])))
fig.add_trace(
    go.Scatter(
        x=x,
        y=fit.eval(x=x),
        name="fit",
        line=dict(color="red"),
        opacity=0.5,
    )
)

components = fit.eval_components(x=x)
for model_name, model_value in list(components.items())[0:2]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        yref="y domain",
        y=0.1,
        showarrow=False,
        text=f'Center: {fit.params[f"{model_name}center"].value:.2f} degrees<br>FWHM: {fit.params[f"{model_name}fwhm"].value:.2f} degrees',
        font=dict(size=18, color=colors[0]),
        bgcolor="white",
    )

fig.show()

## Plot Residuals ##
fig = tr_functions.thesis_fig(
    title=f"MDC",
    xaxis_title=xaxis_title,
    yaxis_title="Residuals",
    equiv_axes=False,
    gridlines=False,
    height=400,
    width=900,
)

fig.add_trace(go.Scatter(x=x, y=row - fit.eval(x=x), name="fit"))
fig.add_hline(y=0, line=dict(dash="dash"))

In [ ]:
params

# Multiple Datset Comparison

In [ ]:
## Average datasets
datas = []
datas = ["Img55.h5", "Img56.h5"]
data_avg = np.mean(datas, axis=0)
print(data_avg.shape)

In [ ]:
ddir = r"E:\atully\arpes_data\2023_May\May13\C60\Imgs"

## HS scans
files = [
    "Img49.h5",
    "Img48.h5",
    "Img50.h5",
    "Img51.h5",
    "Img52.h5",
    "Img53.h5",
    "Img54.h5",
    "Img55.h5",
]

names = [
    "G-K, CT1, 0.4 ps",
    "G-K, CT2, zero delay",
    "G-K, CT1, -5 ps",
    "G-K, CT2, -5 ps",
    "G-K, CT1/CT2, zero delay",
    "K'-G, CT1, 0.4 ps",
    "K'-G, CT2, zero delay",
    "K-M-K, CT1, 0.4 ps",
]

# # data, theta, energy = loading_functions.load_hdf5(ddir, file)

In [ ]:
## Limit Dataset ##

xlim = (-18, 16)
# ylim = (1.815, 2.44)
# ylim = (1.86, 2.49)
# ylim = (1.95, 2.3)
# xlim = None
ylim = None

all_vals = []
for file in files:
    data, theta, energy = loading_functions.load_hdf5(ddir, file)
    all_vals.append(analysis_functions.limit_dataset(theta, energy, data, xlim, ylim))

# x_1, y_1, d_1 = all_vals[0]
# x_8, y_8, d_8 = all_vals[1]

# x_13, y_13, d_13 = all_vals[0]
# # x_14, y_14, d_14 = all_vals[1]
# names = [
#     "CT1",
#     # "CT2",
# ]

# x_1, y_1, d_1 = all_vals[0]
# x_8, y_8, d_8 = all_vals[1]
# x_12, y_12, d_12 = all_vals[2]

# names = [
#     "gamma_k: 1 mW",
#     "gamma_k: pump blocked",
#     "gamma_k: 3.75 mW",
# ]

In [ ]:
## Get and Plot 1D Data --> EDCs & MDCs ##

no_scans = 1

fig = tr_functions.thesis_fig(
    title=f"EDC",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=1000,
)

for i in range(no_scans):
    x, y, d = all_vals[i]
    y, col = tr_functions.get_1d_x_slice(x=x, y=y, data=d, ylims=None, x_range=None)

    # Plot Data
    color = colors[i % len(colors)]
    fig.add_trace(go.Scatter(x=y, y=col, name=names[i], line=dict(color=color)))

fig.show()

## Get and Plot 1D Data --> MDC ##

fig = tr_functions.thesis_fig(
    title=f"MDC",
    xaxis_title=xaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=1000,
)

for i in range(no_scans):
    x, y, d = all_vals[i]
    x, row = tr_functions.get_1d_y_slice(
        x=x, y=y, data=analysis_functions.norm_data(d), xlims=None, y_range=None
    )

    # Plot Data
    color = colors[i % len(colors)]
    fig.add_trace(go.Scatter(x=x, y=row, name=names[i], line=dict(color=color)))

fig.update_layout(showlegend=True)

fig.show()